
### Session state artifacts (in-memory)
This notebook uses `InMemorySessionService`, so state persists only during the notebook runtime for a given `user_id` + `session_id`.

Common keys you’ll see:
- Child profile:
  - `user:child:name`, `user:child:age_years`, `user:child:diagnosis`
- Per-category (category key uses spaces → underscores):
  - `delay:{cat}:months`
  - `pending_questions:{cat}`
  - `qna_idx:{cat}`
  - `qna_answers:{cat}`
  - `dev_age:{cat}:months`
  - `focus_milestones:{cat}`
  - `activity_plan:{cat}` (raw +/or structured plan content, depending on tool output)

---

## How to run (typical flow)
1) Run cells top-to-bottom (loads CDC table, defines tools/agents, initializes runners).
2) Start Q&A in a single session:
   - Use `questions_runner.run_debug(...)` with `user_id` + `session_id`.
3) Re-run Q&A steps as you answer each question (yes/no/not sure).
4) When ready, generate the plan:
   - Use `plan_runner.run_debug(...)` with the **same** `user_id` + `session_id`.
5) Edit the plan by prompting `plan_runner.run_debug(...)` again with constraints.

---

## Limitations / known constraints
- **Hardcoded Excel path:** update `CDC_TABLE_PATH` for your machine.
- **In-memory persistence:** everything resets if the kernel restarts.
- **Heuristic dev-age update:** developmental age is inferred from parent answers using a simple rule (max “yes” month).
- **Not clinical guidance:** output is supportive planning content, not a prescription.

In [ ]:
# ============================================================
# Genex Therapy Agents (Option B: Q&A-driven developmental age)
# ============================================================

# This notebook defines a *separate* therapy pipeline from research pipeline.
# It:
#   1) Stores child profile in ADK session state
#   2) Estimates category-specific developmental delay (months) using GPT
#   3) Uses the delay + CDC milestone table to generate one-at-a-time questions
#   4) Updates the child's *developmental age* based on parent answers
#   5) Selects "next-step" milestones just above that developmental age
#   6) Generates a 5-day home activity plan
#
# The core idea: the *delay estimate* gives a starting point, but the *true*
# developmental age is refined via parent Q&A, then used for therapy planning.

# -------------------------
# 0. Imports & configuration
# -------------------------

import os
import re
import json
import statistics
from typing import Any, Dict, List

import pandas as pd

from openai import OpenAI

# --- ADK imports (compatible across versions) ---

from google.adk.tools import FunctionTool
from google.adk.tools.tool_context import ToolContext
from google.adk.agents import LlmAgent, SequentialAgent
from google.adk.runners import Runner

# App import moved between ADK versions
try:
    from google.adk.app import App            # newer
except ModuleNotFoundError:
    from google.adk.apps import App           # older/common

# InMemorySessionService moved between ADK versions
try:
    from google.adk.services.session_service import InMemorySessionService  # newer
except ModuleNotFoundError:
    from google.adk.sessions import InMemorySessionService                  # older/common

from google.adk.models.lite_llm import LiteLlm


# ---------- API clients ----------

# Make sure your OPENAI_API_KEY is set in the environment before running this cell.
openai_client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

# ADK LLM wrapper (LiteLlm uses the 'litellm' package under the hood)
# We will use gpt-4o-mini for all agents in this notebook.
gpt_model = LiteLlm(model="gpt-4o-mini")

# Simple retry config for agents (optional – ADK will also retry on some errors)
retry_config = {
    "max_retries": 2,
}

# Single shared session service for all apps in this notebook
session_service = InMemorySessionService()

THERAPY_APP_NAME = "genex_therapy_app"

In [2]:
# -------------------------------
# 1. CDC milestone table loader
# -------------------------------

# Path to your CDC milestone Excel file
CDC_TABLE_PATH = r"C:/Users/T490/Downloads/Genetics-Dashboard/milestone-cdc-table.xlsx"

cdc_df = pd.read_excel(CDC_TABLE_PATH)
cdc_df.columns = cdc_df.columns.str.strip()

# Normalize columns
cdc_df["category_clean"] = cdc_df["category"].astype(str).str.strip().str.lower()
cdc_df["milestone_clean"] = cdc_df["milestone"].astype(str).str.strip()
cdc_df["months"] = pd.to_numeric(cdc_df["months"], errors="coerce")

CDC_AGES = sorted(cdc_df["months"].dropna().unique())

# Logical category mapping – you can extend this for other domains
CATEGORY_MAP: Dict[str, str] = {
    "gross_motor": "movement and physical",
    "physical": "movement and physical",
    "physical & movement": "movement and physical",
    "movement_and_physical": "movement and physical",
    "movement and physical": "movement and physical",
    "social_emotional": "social and emotional",
    "social & emotional": "social and emotional",
    "language_communication": "language and communication",
    "language & communication": "language and communication",
    "cognitive": "cognitive",
}


In [3]:
# -------------------------------
# 2. Child profile tools
# -------------------------------

def save_child_profile(tool_context: ToolContext, name: str, age_years: int, diagnosis: str) -> Dict[str, Any]:
    """
    Save or update the child's profile in ADK session state.

    Keys:
      - user:child:name
      - user:child:age_years
      - user:child:diagnosis
    """
    name = name.strip()
    diagnosis = diagnosis.strip()

    tool_context.state["user:child:name"] = name
    tool_context.state["user:child:age_years"] = int(age_years)
    tool_context.state["user:child:diagnosis"] = diagnosis

    return {
        "status": "success",
        "name": name,
        "age_years": int(age_years),
        "diagnosis": diagnosis,
        "message": f"Stored child profile: {name}, {age_years} years, diagnosis: {diagnosis}.",
    }


def retrieve_child_profile(tool_context: ToolContext) -> Dict[str, Any]:
    """
    Retrieve the child's profile from ADK session state.
    """
    name = tool_context.state.get("user:child:name")
    age_years = tool_context.state.get("user:child:age_years")
    diagnosis = tool_context.state.get("user:child:diagnosis")

    if not name or age_years is None or not diagnosis:
        return {
            "status": "error",
            "message": "Child profile is missing. Please provide name, age (in years), and diagnosis.",
        }

    return {
        "status": "success",
        "name": name,
        "age_years": age_years,
        "diagnosis": diagnosis,
    }


save_child_profile_tool = FunctionTool(func=save_child_profile)
retrieve_child_profile_tool = FunctionTool(func=retrieve_child_profile)



In [4]:
# -------------------------------
# 3. Delay estimator tool (GPT-only)
# -------------------------------

def _extract_delay_months_from_text(text: str) -> int:
    """
    Extract the first integer from a GPT response.
    Falls back to 0 if nothing is found.
    """
    if not text:
        return 0
    first_line = text.strip().splitlines()[0]
    m = re.search(r"\d+", first_line)
    if not m:
        return 0
    return int(m.group(0))


def estimate_delay_gpt(tool_context: ToolContext, category: str) -> Dict[str, Any]:
    """
    Estimate developmental delay (in months) for the given category using GPT.

    - If a delay is already stored in state for this category, re-use it
      and avoid another API call.
    - Otherwise, call GPT once to get a numeric delay estimate + short explanation.
    """
    # 1) Check if delay already in state
    cat_norm = CATEGORY_MAP.get(category, category).lower()
    key = cat_norm.replace(" ", "_")

    existing = tool_context.state.get(f"delay:{key}:months")
    if isinstance(existing, (int, float)) and existing > 0:
        return {
            "status": "success",
            "category": cat_norm,
            "delay_months": int(existing),
            "source": "state_cache",
            "raw_text": {},
        }

    # 2) Need child profile
    profile = retrieve_child_profile(tool_context)
    if profile.get("status") != "success":
        return {
            "status": "error",
            "message": "Child profile is missing. Please provide name, age, and diagnosis first.",
        }

    name = profile["name"]
    age_years = profile["age_years"]
    diagnosis = profile["diagnosis"]

    prompt = f"""
You are a pediatric developmental specialist.

Child:
- Name: {name}
- Age: {age_years} years
- Diagnosis: {diagnosis}

Question:
For children ages 0–5 with {diagnosis}, what is a **typical average developmental delay**
in the domain "{cat_norm}" compared to neurotypical peers?

Respond in this exact format:
FIRST LINE: a single integer number of months (e.g. 12)
THEN: one short sentence explaining the reasoning.
""".strip()

    try:
        completion = openai_client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "user", "content": prompt}],
        )
        text = completion.choices[0].message.content
    except Exception as e:
        return {
            "status": "error",
            "message": f"OpenAI error while estimating delay: {e}",
        }

    delay_months = _extract_delay_months_from_text(text)
    if delay_months <= 0:
        delay_months = 12  # conservative fallback

    # Store in state
    tool_context.state[f"delay:{key}:months"] = delay_months

    return {
        "status": "success",
        "category": cat_norm,
        "delay_months": delay_months,
        "source": "gpt",
        "raw_text": {"gpt": text},
    }


delay_estimator_tool = FunctionTool(func=estimate_delay_gpt)


In [ ]:
# -------------------------------
# 4. Milestone question builder
# -------------------------------

def build_milestone_questions(
    tool_context: ToolContext,
    category: str,
    window_months: int = 12,
) -> Dict[str, Any]:
    """
    Use child's chronological age + stored delay to decide which milestone
    ages to question, and return a *list* of questions, but also initialize
    one-at-a-time Q&A state:

    - pending_questions:{cat_key} : list of question dicts
    - question_index:{cat_key}    : current question index (0-based)
    - answers:{cat_key}           : dict of question_id -> {"months": int, "answer": str}

    This tool returns the *first* question string so the agent can ask it.
    """
    # Child profile
    profile = retrieve_child_profile(tool_context)
    if profile.get("status") != "success":
        return {"status": "error", "message": "Child profile missing."}

    age_years = profile["age_years"]
    chrono_months = int(age_years * 12)

    cat_norm = CATEGORY_MAP.get(category, category).lower()
    key = cat_norm.replace(" ", "_")

    # Developmental delay from state (default 12 months if missing)
    delay_months = tool_context.state.get(f"delay:{key}:months", 12)

    # Rough developmental age estimate
    approx_dev_months = max(0, chrono_months - delay_months)

    # Choose a window around this estimate
    min_months = max(2, approx_dev_months - window_months // 2)
    max_months = min(60, approx_dev_months + window_months // 2)

    relevant_ages = [m for m in CDC_AGES if min_months <= m <= max_months]
    if not relevant_ages:
        relevant_ages = [min(CDC_AGES), max(CDC_AGES)]

    subset = cdc_df[
        (cdc_df["category_clean"] == cat_norm)
        & (cdc_df["months"].isin(relevant_ages))
    ].copy()

    if subset.empty:
        return {
            "status": "error",
            "message": f"No milestones found for category '{cat_norm}' in the selected age window.",
        }

    questions: List[Dict[str, Any]] = []
    for i, row in subset.iterrows():
        q_id = f"{cat_norm}_{row['months']}_{i}"
        questions.append(
            {
                "id": q_id,
                "months": int(row["months"]),
                "question": (
                    f"Can {profile['name']} "
                    f"'{row['milestone_clean']}'right now? (yes/no/not sure)"
                ),
            }
        )

    # Initialize Q&A state
    tool_context.state[f"pending_questions:{key}"] = questions
    tool_context.state[f"question_index:{key}"] = 0
    tool_context.state[f"answers:{key}"] = {}

    first_q = questions[0]["question"]

    return {
        "status": "success",
        "approx_dev_months": approx_dev_months,
        "chrono_months": chrono_months,
        "delay_months": delay_months,
        "first_question": first_q,
        "total_questions": len(questions),
    }


milestone_questions_tool = FunctionTool(func=build_milestone_questions)

In [ ]:
# -------------------------------
# 5. Q&A answer scoring tool
# -------------------------------

def _normalize_yes_no(answer_text: str) -> str:
    """
    Map free-form parent answer to 'yes' / 'no' / 'not sure'.
    """
    if not answer_text:
        return "not sure"

    t = answer_text.strip().lower()

    # very simple heuristics; you can extend
    if any(w in t for w in ["yes", "yeah", "yep", "she does", "he does", "can do"]):
        return "yes"
    if any(w in t for w in ["no", "not yet", "can't", "cannot", "doesn't", "does not"]):
        return "no"
    if any(w in t for w in ["not sure", "maybe", "sometimes", "half", "in between"]):
        return "not sure"

    return "not sure"


def _compute_dev_age_from_answers(answers: Dict[str, Dict[str, Any]]) -> int:
    """
    Simple heuristic:
      - Collect 'months' for all questions with answer == 'yes'
      - If none, dev-age = minimum of all months
      - Otherwise, dev-age = max months where answer == 'yes'
    """
    if not answers:
        return 6  # very conservative fallback

    months_all = [v["months"] for v in answers.values()]
    months_yes = [v["months"] for v in answers.values() if v["answer"] == "yes"]

    if months_yes:
        return int(max(months_yes))

    return int(min(months_all))


def answer_milestone_question(tool_context: ToolContext, category: str, parent_answer: str):
    key = category.replace(" ", "_")

    questions = tool_context.state.get(f"pending_questions:{key}", [])
    idx = tool_context.state.get(f"qna_idx:{key}", 0)

    # --- Safety ---
    if idx >= len(questions):
        return {
            "status": "done",
            "message": "All questions completed."
        }

    # --- Record answer ---
    current_q = questions[idx]
    answers = tool_context.state.get(f"qna_answers:{key}", [])
    answers.append({
        "question": current_q["milestone"],
        "months": current_q["months"],
        "answer": parent_answer.lower().strip()
    })
    tool_context.state[f"qna_answers:{key}"] = answers

    # --- Move index forward EXACTLY ONCE ---
    idx += 1
    tool_context.state[f"qna_idx:{key}"] = idx

    # --- If more questions remain, ask next ---
    if idx < len(questions):
        next_q = questions[idx]
        return {
            "status": "ask_next",
            "question_number": idx + 1,  # ✅ CORRECT
            "question_text": next_q["question"],
            "answered_question": current_q["question"],
            "parent_answer": parent_answer,
        }

    # --- Otherwise finish ---
    dev_age_months = compute_dev_age_from_answers(answers)
    tool_context.state[f"dev_age:{key}:months"] = dev_age_months

    return {
        "status": "done",
        "dev_age_months": dev_age_months,
        "answered_question": current_q["question"],
        "parent_answer": parent_answer,
    }

answer_milestone_question_tool = FunctionTool(func=answer_milestone_question)


In [7]:
# -------------------------------
# 6. Next-step milestones + 5-day plan tools
# -------------------------------

def select_next_milestones(
    tool_context: ToolContext,
    category: str,
    max_milestones: int = 6,
) -> Dict[str, Any]:
    """
    Choose 3–6 CDC milestones immediately above the child's developmental age
    for the given category.

    Expects in state:
        dev_age:{category_key}:months = <int>
    """
    profile = retrieve_child_profile(tool_context)
    if profile.get("status") != "success":
        return {"status": "error", "message": "Child profile missing."}

    cat_norm = CATEGORY_MAP.get(category, category).lower()
    key = cat_norm.replace(" ", "_")

    dev_age_months = tool_context.state.get(f"dev_age:{key}:months", None)

    if dev_age_months is None:
        return {
            "status": "error",
            "message": f"No developmental age stored for category '{cat_norm}'. "
                       "Finish the milestone Q&A first.",
        }

    # Define the “next step” range (just above current level)
    min_m = dev_age_months + 1
    max_m = dev_age_months + 12

    subset = cdc_df[
        (cdc_df["category_clean"] == cat_norm)
        & (cdc_df["months"] >= min_m)
        & (cdc_df["months"] <= max_m)
    ].sort_values("months")

    if subset.empty:
        return {
            "status": "error",
            "message": f"No next-step milestones found for '{cat_norm}'.",
            "milestones": [],
        }

    milestones: List[Dict[str, Any]] = []
    for i, row in subset.iterrows():
        milestones.append(
            {
                "id": f"{cat_norm}_{row['months']}_{i}",
                "months": int(row["months"]),
                "milestone": row["milestone_clean"],
            }
        )

    selected = milestones[:max_milestones]

    # Store for the activity planner
    tool_context.state[f"focus_milestones:{key}"] = selected

    return {
        "status": "success",
        "dev_age": dev_age_months,
        "category": cat_norm,
        "selected_milestones": selected,
    }


def gpt_generate_activity_plan(tool_context: ToolContext, category: str) -> Dict[str, Any]:
    """
    Generate a 5-day activity plan targeting the selected milestones.

    Expects:
        state["focus_milestones:{category_key}"] from select_next_milestones()
    Uses GPT-4o-mini via openai_client.
    """
    cat_norm = CATEGORY_MAP.get(category, category).lower()
    key = cat_norm.replace(" ", "_")

    profile = retrieve_child_profile(tool_context)
    if profile.get("status") != "success":
        return {"status": "error", "message": "Child profile missing."}

    name = profile["name"]
    age = profile["age_years"]
    diag = profile["diagnosis"]

    milestones = tool_context.state.get(f"focus_milestones:{key}", None)
    if not milestones:
        return {
            "status": "error",
            "message": f"No selected milestones found for category '{cat_norm}'. "
                       f"Run the next-step milestone tool first.",
        }

    milestone_list = "\n".join(
        f"- ({m['months']} months) {m['milestone']}" for m in milestones
    )

    prompt = f"""
You are a pediatric therapist helping parents at home.

Child:
- Name: {name}
- Age: {age} years
- Diagnosis: {diag}
- Target category: {cat_norm}

Target “next step” milestones:
{milestone_list}

TASK:
Create a **5-day home activity plan**, with 1–2 activities per day,
directly targeting these milestones.

REQUIREMENTS:
- Use warm, parent-friendly language.
- Make each activity short, practical, play-based, and doable at home.
- Avoid medical jargon or mentioning therapy models.
- Do NOT mention that you are an AI or that this is a prompt.

FORMAT:
Return in this exact JSON-style structure (no extra commentary):

{{
  "day_1": ["activity 1", "activity 2 optional"],
  "day_2": ["..."],
  "day_3": ["..."],
  "day_4": ["..."],
  "day_5": ["..."]
}}
""".strip()

    try:
        completion = openai_client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "user", "content": prompt}],
        )
        content = completion.choices[0].message.content
    except Exception as e:
        return {"status": "error", "message": f"OpenAI error while generating plan: {e}"}

    # Try to parse JSON-ish content just for convenience (not required)
    plan_parsed: Dict[str, Any] = {}
    try:
        plan_parsed = json.loads(content)
    except Exception:
        # If it's not valid JSON, we still return the raw content
        plan_parsed = {}

    # Stash raw plan in state for UI reuse
    tool_context.state[f"activity_plan:{key}"] = content

    return {
        "status": "success",
        "category": cat_norm,
        "plan_raw": content,
        "plan_parsed": plan_parsed,
    }


focus_milestones_tool = FunctionTool(func=select_next_milestones)
activity_plan_tool = FunctionTool(func=gpt_generate_activity_plan)



In [8]:
# -------------------------------
# 7. Agents (profile, delay, Q&A, focus, plan, summary)
# -------------------------------

# -- Profile agent (gross motor) --

profile_agent_gross_motor = LlmAgent(
    name="profile_agent_gross_motor",
    model=gpt_model,
    tools=[save_child_profile_tool, retrieve_child_profile_tool],
    description="Collects and stores the child's profile (name, age, diagnosis).",
    instruction="""
You are responsible for ensuring the child's profile is stored.

If the profile is missing or incomplete:
  - Extract the child's name, age in years, and diagnosis from the parent's message.
  - Call `save_child_profile` once with those values.
Then, briefly confirm what you stored.

If the profile is already present and the parent doesn't update it:
  - Call `retrieve_child_profile` to double-check.
  - Respond with a short confirmation like:
      "I already have Emma's profile: 3 years old with Down syndrome."

Do NOT do any milestone or therapy work; other agents handle that.
""",
)


# -- Delay agent (gross motor) --

delay_agent_gross_motor = LlmAgent(
    name="delay_agent_gross_motor",
    model=gpt_model,
    tools=[delay_estimator_tool],
    description="Gets a GPT-based developmental delay estimate (months) for gross motor skills.",
    instruction="""
You estimate developmental delay for the **gross motor** domain
(called "movement and physical" in the CDC table).

Rules:
- ALWAYS call `estimate_delay_gpt` once with category "movement and physical".
- If the tool says the delay is already stored in state, just explain that value.
- Briefly explain:
    * the delay in months
    * that this is a typical average for children with this diagnosis.
- Do NOT ask milestone questions or suggest activities; other agents handle that.
""",
)


# -- Q&A agent (gross motor, one question per turn) --

milestone_qna_agent_gross_motor = LlmAgent(
    name="milestone_qna_agent_gross_motor",
    model=gpt_model,
    tools=[milestone_questions_tool, answer_milestone_question_tool],
    description="Asks one CDC milestone question at a time and refines developmental age.",
    instruction="""
You help parents answer CDC milestone questions for the **gross motor** domain
(category: "movement and physical") ONE QUESTION AT A TIME.

There are two situations:

(1) FIRST TIME (no questions yet):
    - Detect this by noticing that there is no entry in state for
      "pending_questions:movement_and_physical".
    - Call `build_milestone_questions` with category "movement and physical".
    - From the tool result, take `first_question` and present it to the parent,
      clearly marking it as "Question 1".
    - Ask the parent to answer with yes/no/not sure.
    - Do NOT call `answer_milestone_question` yet in this first turn.

(2) FOLLOW-UP TURNS (parent is answering):
    - The parent will reply to the most recent question.
    - Extract their answer as a short phrase like "yes", "no", or "not sure".
    - Call `answer_milestone_question` with:
        - category = "movement and physical"
        - parent_answer = that short phrase
    - If the tool returns status = "ask_next":
        - Present the `next_question` text clearly (e.g., "Question 3: ...").
        - Ask the parent again to answer yes/no/not sure.
        - Mention how many questions you've covered so far if available.
    - If the tool returns status = "done":
        - Use `dev_age_months` to say something like:
            "Based on your answers, Emma's gross motor skills are close to
             the 15–18 month level."
        - Explain this in warm, parent-friendly language.
        - DO NOT suggest specific activities here; that will come later.

In all cases:
- Keep your messages short and clear.
- Only ask ONE milestone question at a time.
- Do not call tools unrelated to milestone Q&A.
""",
)


# -- Focus agent: pick next-step milestones --

focus_agent_gross_motor = LlmAgent(
    name="focus_agent_gross_motor",
    model=gpt_model,
    tools=[focus_milestones_tool],
    description="Chooses 3–6 next-step milestones for gross motor skills.",
    instruction="""
You select next-step CDC milestones for the **gross motor** category
("movement and physical") AFTER a developmental age has been estimated.

- Always call `select_next_milestones` with category "movement and physical".
- If the tool returns an error saying no developmental age is stored,
  respond briefly:
    "We need to finish the milestone Q&A first before selecting next-step goals."
  and do nothing else.
- If it succeeds:
    - Briefly list the selected milestones (2–4 lines is enough),
      highlighting why they are good next-step goals.
""",
)


# -- Activity plan agent: 5-day plan --

activity_agent_gross_motor = LlmAgent(
    name="activity_agent_gross_motor",
    model=gpt_model,
    tools=[activity_plan_tool],
    description="Builds a 5-day home activity plan for gross motor next-step milestones.",
    instruction="""
You generate a 5-day activity plan for the **gross motor** category
("movement and physical").

- Always call `gpt_generate_activity_plan` with category "movement and physical".
- If the tool returns an error (e.g., no milestones set yet),
  explain briefly what needs to happen first (finish Q&A and next-step goals).
- If it succeeds:
    - Read the returned plan (either `plan_parsed` if valid JSON or `plan_raw`).
    - Present it to the parent as:
        Day 1:
          - activity...
        Day 2:
          - ...
      in warm, parent-friendly language.
""",
)


# -- Final summary agent --

milestone_summary_agent_gross_motor = LlmAgent(
    name="milestone_summary_agent_gross_motor",
    model=gpt_model,
    tools=[retrieve_child_profile_tool],
    description="Summarizes delay, developmental age, milestones, and plan for parents.",
    instruction="""
You write a short, parent-friendly recap of Emma's gross motor plan.

Steps:
1. Call `retrieve_child_profile` to get name, age, diagnosis.
2. Read from state:
   - delay:movement_and_physical:months
   - dev_age:movement_and_physical:months
   - focus_milestones:movement_and_physical
   - activity_plan:movement_and_physical (raw text is fine)

3. Structure your summary as:

   Line 1:
     "[NAME] is [AGE] years old and has [DIAGNOSIS]."

   Paragraph:
     - Explain the estimated delay in gross motor skills (in months).
     - Explain the approximate developmental age level in plain language
       (e.g., "around the 12–15 month level").

   Section "This week's focus":
     - Bullet 3–4 key milestones chosen as next-step goals.

   Section "5-day home plan":
     - Briefly summarize what happens on each day (1–2 bullets per day),
       using warm, encouraging language.
     - You can loosely paraphrase from the plan stored in state; no need
       to preserve exact wording.

4. Final note:
     - Include a gentle reminder that this does not replace professional
       medical or therapy advice.

If some information is missing (e.g., no plan yet), just summarize what
is available and say what the next step will be.
""",
)



In [9]:
# -------------------------------
# 8. Pipelines & Runners
# -------------------------------

# A) Question pipeline:
#    profile -> delay -> milestone Q&A (multi-turn, one question per run)

questions_pipeline_gross_motor = SequentialAgent(
    name="gross_motor_questions_pipeline",
    sub_agents=[
        profile_agent_gross_motor,
        delay_agent_gross_motor,
        milestone_qna_agent_gross_motor,
    ],
)

questions_app = App(
    name=THERAPY_APP_NAME,
    root_agent=questions_pipeline_gross_motor,
)

questions_runner = Runner(
    app=questions_app,
    session_service=session_service,
)

print("Questions pipeline initialized.")
print(f"  App name: {questions_app.name}")


# B) Plan pipeline:
#    focus milestones -> activity plan -> parent summary

plan_pipeline_gross_motor = SequentialAgent(
    name="gross_motor_plan_pipeline",
    sub_agents=[
        focus_agent_gross_motor,
        activity_agent_gross_motor,
        milestone_summary_agent_gross_motor,
    ],
)

plan_app = App(
    name=THERAPY_APP_NAME,  # same app name so it shares session state
    root_agent=plan_pipeline_gross_motor,
)

plan_runner = Runner(
    app=plan_app,
    session_service=session_service,
)

print("Plan pipeline initialized.")
print(f"  App name: {plan_app.name}")


Questions pipeline initialized.
  App name: genex_therapy_app
Plan pipeline initialized.
  App name: genex_therapy_app


In [10]:
# -------------------------------
# 9. Helper: pretty-print latest model response
# -------------------------------

def print_last_model_message(events: List[Any]) -> None:
    """
    Utility to show the last model-authored text in a run_debug trace.
    """
    last_text = None
    for ev in events:
        if getattr(ev, "content", None):
            for part in getattr(ev.content, "parts", []):
                if getattr(part, "text", None):
                    last_text = part.text
    if last_text:
        print(last_text)
    else:
        print("(No model text found in events)")


In [26]:
import json
from typing import Dict, Any, Optional
from google.adk.tools import FunctionTool
from google.adk.tools.tool_context import ToolContext
from google.adk.agents import LlmAgent

# -----------------------------
# Plan editor tool (state-based)
# -----------------------------

PLAN_KEY_TEMPLATE = "activity_plan:{key}"  # e.g. activity_plan:movement_and_physical

def _cat_key(category: str) -> str:
    # must match how you store other keys
    cat_norm = CATEGORY_MAP.get(category, category).lower()
    return cat_norm.replace(" ", "_")

def get_activity_plan(tool_context: ToolContext, category: str) -> Dict[str, Any]:
    key = _cat_key(category)
    plan_key = PLAN_KEY_TEMPLATE.format(key=key)

    plan_raw = tool_context.state.get(plan_key)
    if not plan_raw:
        return {"status": "error", "message": f"No plan found in state under '{plan_key}'."}

    # Try to parse JSON if possible
    plan_parsed: Optional[dict] = None
    try:
        plan_parsed = json.loads(plan_raw) if isinstance(plan_raw, str) else plan_raw
    except Exception:
        plan_parsed = None

    return {
        "status": "success",
        "category": category,
        "plan_key": plan_key,
        "plan_raw": plan_raw,
        "plan_parsed": plan_parsed,
    }

def save_activity_plan(tool_context: ToolContext, category: str, plan_raw: str) -> Dict[str, Any]:
    key = _cat_key(category)
    plan_key = PLAN_KEY_TEMPLATE.format(key=key)

    tool_context.state[plan_key] = plan_raw
    return {"status": "success", "message": f"Updated plan saved to '{plan_key}'.", "plan_key": plan_key}

def edit_activity_plan(tool_context: ToolContext, category: str, user_request: str) -> Dict[str, Any]:
    """
    Reads the existing plan from state, asks GPT to apply the user’s edit request
    (e.g. shorten Day 2, replace an activity, make it more fun), then saves back to state.
    """
    # 1) Load profile (optional but helpful)
    profile = retrieve_child_profile(tool_context)
    name = profile.get("name", "the child") if profile.get("status") == "success" else "the child"
    diagnosis = profile.get("diagnosis", "") if profile.get("status") == "success" else ""

    # 2) Load existing plan
    current = get_activity_plan(tool_context, category)
    if current.get("status") != "success":
        return current

    plan_raw = current["plan_raw"]

    # 3) Ask GPT to edit (keep same milestones; only modify what user requested)
    prompt = f"""
You are editing a parent-friendly 5-day home activity plan.

Child:
- Name: {name}
- Diagnosis: {diagnosis}
Category: {category}

USER REQUEST:
{user_request}

CURRENT PLAN (may be JSON or text):
{plan_raw}

EDITING RULES:
- Keep it a 5-day plan (Day 1..Day 5).
- Preserve the intent: still targets the same developmental goals.
- Change ONLY what the user asked (e.g., shorten Day 2; replace Day 3).
- Keep it practical, fun, and doable at home.
- Output format:
  If the input plan looks like JSON, return valid JSON.
  Otherwise return clean text with "Day 1:" ... "Day 5:".
"""

    try:
        resp = openai_client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "user", "content": prompt}],
        )
        edited = resp.choices[0].message.content.strip()
    except Exception as e:
        return {"status": "error", "message": f"GPT edit error: {e}"}

    # 4) Save updated plan to state
    save_result = save_activity_plan(tool_context, category, edited)

    return {
        "status": "success",
        "category": category,
        "plan_key": current["plan_key"],
        "updated_plan": edited,
        "save_result": save_result,
    }

# Wrap as ADK tools
get_activity_plan_tool  = FunctionTool(func=get_activity_plan)
edit_activity_plan_tool = FunctionTool(func=edit_activity_plan)

# -----------------------------
# Plan editor agent
# -----------------------------

plan_editor_agent_gross_motor = LlmAgent(
    name="plan_editor_agent_gross_motor",
    model=gpt_model,
    tools=[edit_activity_plan_tool],
    description="Edits the existing 5-day plan based on parent follow-up requests.",
    instruction="""
You are a plan editor.

- ALWAYS call `edit_activity_plan` once.
- category MUST be "movement and physical".
- user_request should be the parent's message verbatim (or a clean summary).
- After tool returns, show the updated plan to the parent clearly.
- Do NOT regenerate a brand-new plan unless user explicitly asks “start over”.
""",
)


In [11]:
# -------------------------------
# 10. HOW TO RUN (example flows)
# -------------------------------
# You can copy-paste these cells into your notebook and run interactively.
#
# --- First turn: profile + delay + FIRST milestone question ---

from datetime import datetime

session_id = "emma-gross-001"

events = await questions_runner.run_debug(
    "My child's name is Emma. She is 3 years old and has Down syndrome. "
    "Please assess her gross motor skills.",
    user_id="emma_parents",
    session_id=session_id,
    quiet=True,
    verbose=False,
)

print_last_model_message(events)

# This should:
#   - store Emma's profile,
#   - estimate a gross-motor delay in months,
#   - and then ask **Question 1** (one CDC milestone).
#

To assess Emma's gross motor skills, let's start with the first question.

**Question 1:** At around 6 months, does Emma "roll from tummy to back"? (yes/no/not sure) 

Please answer with yes, no, or not sure.


In [12]:
# --- Follow-up turns: answer one question at a time ---
#
# For each new answer, call the same questions_runner with the SAME session_id.
#
# Example for the second turn (answering Question 1):
#
events = await questions_runner.run_debug(
    "Yes, she can do that.",
    user_id="emma_parents",
    session_id=session_id,
    quiet=True,
    verbose=False,
)
print_last_model_message(events)

# The Q&A agent will:
#   - record this answer,
#   - either ask the next question (status='ask_next'),
#   - or, once it has enough answers, compute a developmental age
#     and tell you it's done.
#
# You continue this pattern until the agent says something like:
#     "Based on your answers, Emma's gross motor skills are around
#      the 15–18 month level."

You answered "yes" to the question about whether Emma can sit without support at around 9 months.

**Question 3:** At around 6 months, does Emma "push up with straight arms when on tummy"? (yes/no/not sure) 

Please answer with yes, no, or not sure.


In [13]:
events = await questions_runner.run_debug(
    "No",
    user_id="emma_parents",
    session_id=session_id,
    quiet=True,
    verbose=False,
)
print_last_model_message(events)


You answered "no" to the question about whether Emma can stand up without holding on at around 12 months.

**Question 5:** At around 6 months, does Emma "lean on hands to support herself when sitting"? (yes/no/not sure) 

Please answer with yes, no, or not sure.


In [14]:
events = await questions_runner.run_debug(
    "yes",
    user_id="emma_parents",
    session_id=session_id,
    quiet=True,
    verbose=False,
)
print_last_model_message(events)


You answered "no" to the question about whether Emma can walk using furniture to support herself at around 12 months.

**Question 7:** At around 9 months, does Emma "get to a sitting position by herself"? (yes/no/not sure) 

Please answer with yes, no, or not sure.


In [15]:
events = await questions_runner.run_debug(
    "yes",
    user_id="emma_parents",
    session_id=session_id,
    quiet=True,
    verbose=False,
)
print_last_model_message(events)

You answered "no" to the question about whether Emma can crawl forward on her belly at around 12 months.

**Question 9:** At around 9 months, does Emma "move things from one hand to her other hand"? (yes/no/not sure) 

Please answer with yes, no, or not sure.


In [16]:
events = await questions_runner.run_debug(
    "not sure",
    user_id="emma_parents",
    session_id=session_id,
    quiet=True,
    verbose=False,
)
print_last_model_message(events)

You answered "no" to the question about whether Emma can pull up to stand at around 12 months.

**Question 11:** At around 9 months, does Emma "use her fingers to rake food towards herself"? (yes/no/not sure) 

Please answer with yes, no, or not sure.


In [17]:
events = await questions_runner.run_debug(
    "yes",
    user_id="emma_parents",
    session_id=session_id,
    quiet=True,
    verbose=False,
)
print_last_model_message(events)

You answered "no" to the question about whether Emma can walk a few steps without holding on at around 12 months.

**Question 13:** At around 9 months, does Emma "sit without support"? (yes/no/not sure) 

Please answer with yes, no, or not sure.


In [18]:
events = await questions_runner.run_debug(
    "yes",
    user_id="emma_parents",
    session_id=session_id,
    quiet=True,
    verbose=False,
)
print_last_model_message(events)

You answered "no" to the question about whether Emma can squat to pick up a toy at around 12 months.

**Question 15:** At around 12 months, does Emma "pull up to stand"? (yes/no/not sure) 

Please answer with yes, no, or not sure.


In [19]:
events = await questions_runner.run_debug(
    "yes",
    user_id="emma_parents",
    session_id=session_id,
    quiet=True,
    verbose=False,
)
print_last_model_message(events)

You answered "no" to the question about whether Emma can walk while holding on to furniture at around 12 months.

Based on your answers, Emma's gross motor skills are close to the 6-month level. This suggests that she may have developmental delays in gross motor skills as expected for her age. It's important to remember that every child develops at their own pace, and this assessment helps us understand where she is in her development.

If you have any questions or need further assistance, feel free to ask!


In [20]:
# --- Once Q&A is done: generate plan & summary ---
#
# Then you call the PLAN pipeline runner (same session_id) once:
#
events = await plan_runner.run_debug(
    "Please create a 5-day home plan for Emma's gross motor skills.",
    user_id="emma_parents",
    session_id=session_id,
    quiet=True,
    verbose=False,
)
print_last_model_message(events)

# This should:
#   - read the stored dev-age,
#   - pick 3–6 next-step milestones,
#   - generate a GPT-based 5-day plan,
#   - and produce a warm parent-facing summary.

Event from an unknown agent: milestone_qna_agent_gross_motor, event id: 4cd02808-5ce9-4330-bf08-d96beff1c4b8
Event from an unknown agent: milestone_qna_agent_gross_motor, event id: a4c2568c-6817-434f-90e7-9617e120656f
Event from an unknown agent: milestone_qna_agent_gross_motor, event id: 872b138e-3a80-44f6-8a8a-aee1515d44dd
Event from an unknown agent: delay_agent_gross_motor, event id: 70671acd-68cd-4375-ab42-3a5652f8e8bc
Event from an unknown agent: profile_agent_gross_motor, event id: fad89639-a01b-4529-9676-16bb814621e2
Event from an unknown agent: profile_agent_gross_motor, event id: 4be549fd-3a24-44d2-8e65-a66789be1478
Event from an unknown agent: profile_agent_gross_motor, event id: c7ab5c8f-4ad0-4cc2-9dbc-78e75083fef7
Event from an unknown agent: milestone_qna_agent_gross_motor, event id: f80c084f-aa24-4eb8-9408-88fb1b61c818
Event from an unknown agent: milestone_qna_agent_gross_motor, event id: 6cfcf035-ba27-48ee-a068-b2d8f81cee94
Event from an unknown agent: milestone_qna_ag

**Emma is 3 years old and has Down syndrome.**

Emma's gross motor skills are estimated to be delayed by about 24 months, meaning she may function around the 12-month level.

### This week's focus:
- Encouraging rolling skills
- Enhancing stability while sitting
- Promoting pulling up to stand
- Encouraging crawling

### 5-day home plan:

**Day 1: Tummy Time and Reaching**
- Engage Emma in tummy time for 10–15 minutes. Place colorful toys just out of reach to encourage her to stretch and reach for them.
- This activity helps strengthen her neck and shoulder muscles, promoting pushing up on her arms.

**Day 2: Rolling and Reaching**
- Encourage Emma to practice rolling from tummy to back and back again. Use fun sound-making or colorful toys to motivate her to roll toward them.
- Help her enhance her rolling skills and make movement enjoyable through play.

**Day 3: Sit and Play**
- Support Emma in a sitting position with pillows around her for safety. Provide her with toys to pick up, s

In [23]:
events = await plan_runner.run_debug(
    "Can you shorten Day 2 to just ONE activity? I’m busy.",
    user_id="emma_parents",
    session_id=session_id,
    quiet=True,
    verbose=False,
)
print_last_model_message(events)

Event from an unknown agent: milestone_qna_agent_gross_motor, event id: 4cd02808-5ce9-4330-bf08-d96beff1c4b8
Event from an unknown agent: milestone_qna_agent_gross_motor, event id: a4c2568c-6817-434f-90e7-9617e120656f
Event from an unknown agent: milestone_qna_agent_gross_motor, event id: 872b138e-3a80-44f6-8a8a-aee1515d44dd
Event from an unknown agent: delay_agent_gross_motor, event id: 70671acd-68cd-4375-ab42-3a5652f8e8bc
Event from an unknown agent: profile_agent_gross_motor, event id: fad89639-a01b-4529-9676-16bb814621e2
Event from an unknown agent: profile_agent_gross_motor, event id: 4be549fd-3a24-44d2-8e65-a66789be1478
Event from an unknown agent: profile_agent_gross_motor, event id: c7ab5c8f-4ad0-4cc2-9dbc-78e75083fef7
Event from an unknown agent: milestone_qna_agent_gross_motor, event id: f80c084f-aa24-4eb8-9408-88fb1b61c818
Event from an unknown agent: milestone_qna_agent_gross_motor, event id: 6cfcf035-ba27-48ee-a068-b2d8f81cee94
Event from an unknown agent: milestone_qna_ag

Event from an unknown agent: profile_agent_gross_motor, event id: e1cd2718-f3ef-4afe-9910-4a42e07d8f03
Event from an unknown agent: profile_agent_gross_motor, event id: 0642c958-ccdf-45ff-9294-369cb7733a5a
Event from an unknown agent: profile_agent_gross_motor, event id: 0f548915-297b-4768-8885-35de4997ae72


**Emma is 3 years old and has Down syndrome.**

Emma's gross motor skills are estimated to be delayed by about 24 months, meaning she may function around the 12-month level.

### This week's focus:
- Encouraging rolling skills
- Enhancing stability while sitting
- Promoting pulling up to stand
- Encouraging crawling

### 5-day home plan:

**Day 1: Tummy Time and Reaching**
- Engage Emma in tummy time for 10–15 minutes. Place colorful toys just out of reach to encourage her to stretch and reach for them.
- This helps strengthen her neck and shoulder muscles, promoting pushing up on her arms.

**Day 2: Rolling**
- **Activity:** Encourage Emma to practice rolling from tummy to back. Use a colorful toy or one that makes sounds to motivate her to roll towards it.
- **Goal:** This focuses on enhancing her rolling skills and making movement enjoyable.

**Day 3: Sit and Play**
- Support Emma in a sitting position with pillows around her for safety. Provide her with toys to pick up, shake, and 

In [24]:
events = await plan_runner.run_debug(
    "Day 3 isn’t doable (no space for an obstacle course). Please replace with something easier indoors.",
    user_id="emma_parents",
    session_id=session_id,
    quiet=True,
    verbose=False,
)
print_last_model_message(events)

Event from an unknown agent: milestone_qna_agent_gross_motor, event id: 4cd02808-5ce9-4330-bf08-d96beff1c4b8
Event from an unknown agent: milestone_qna_agent_gross_motor, event id: a4c2568c-6817-434f-90e7-9617e120656f
Event from an unknown agent: milestone_qna_agent_gross_motor, event id: 872b138e-3a80-44f6-8a8a-aee1515d44dd
Event from an unknown agent: delay_agent_gross_motor, event id: 70671acd-68cd-4375-ab42-3a5652f8e8bc
Event from an unknown agent: profile_agent_gross_motor, event id: fad89639-a01b-4529-9676-16bb814621e2
Event from an unknown agent: profile_agent_gross_motor, event id: 4be549fd-3a24-44d2-8e65-a66789be1478
Event from an unknown agent: profile_agent_gross_motor, event id: c7ab5c8f-4ad0-4cc2-9dbc-78e75083fef7
Event from an unknown agent: milestone_qna_agent_gross_motor, event id: f80c084f-aa24-4eb8-9408-88fb1b61c818
Event from an unknown agent: milestone_qna_agent_gross_motor, event id: 6cfcf035-ba27-48ee-a068-b2d8f81cee94
Event from an unknown agent: milestone_qna_ag

**Emma is 3 years old and has Down syndrome.**

Emma's gross motor skills are estimated to be delayed by about 24 months, meaning she may function around the 12-month level.

### This week's focus:
- Encouraging rolling skills
- Enhancing stability while sitting
- Promoting pulling up to stand
- Encouraging crawling

### 5-day home plan:

**Day 1: Tummy Time and Reaching**
- Engage Emma in tummy time for 10–15 minutes. Place colorful toys just out of reach to encourage her to stretch and reach for them.
- This helps strengthen her neck and shoulder muscles, promoting pushing up on her arms.

**Day 2: Rolling**
- **Activity:** Encourage Emma to practice rolling from tummy to back. Use a colorful toy or one that makes sounds to motivate her to roll towards it.
- **Goal:** This focuses on enhancing her rolling skills and making movement enjoyable.

**Day 3: Sit and Reach**
- **Activity:** Support Emma in a sitting position with pillows around her for safety. Sit across from her and encour

In [25]:
events = await plan_runner.run_debug(
    "Make the whole week more fun and game-like, but still targeting the same milestones.",
    user_id="emma_parents",
    session_id=session_id,
    quiet=True,
    verbose=False,
)
print_last_model_message(events)


Event from an unknown agent: milestone_qna_agent_gross_motor, event id: 4cd02808-5ce9-4330-bf08-d96beff1c4b8
Event from an unknown agent: milestone_qna_agent_gross_motor, event id: a4c2568c-6817-434f-90e7-9617e120656f
Event from an unknown agent: milestone_qna_agent_gross_motor, event id: 872b138e-3a80-44f6-8a8a-aee1515d44dd
Event from an unknown agent: delay_agent_gross_motor, event id: 70671acd-68cd-4375-ab42-3a5652f8e8bc
Event from an unknown agent: profile_agent_gross_motor, event id: fad89639-a01b-4529-9676-16bb814621e2
Event from an unknown agent: profile_agent_gross_motor, event id: 4be549fd-3a24-44d2-8e65-a66789be1478
Event from an unknown agent: profile_agent_gross_motor, event id: c7ab5c8f-4ad0-4cc2-9dbc-78e75083fef7
Event from an unknown agent: milestone_qna_agent_gross_motor, event id: f80c084f-aa24-4eb8-9408-88fb1b61c818
Event from an unknown agent: milestone_qna_agent_gross_motor, event id: 6cfcf035-ba27-48ee-a068-b2d8f81cee94
Event from an unknown agent: milestone_qna_ag

**Emma is 3 years old and has Down syndrome.**

Emma's gross motor skills are estimated to be delayed by about 24 months, meaning she may function around the 12-month level.

### This week's focus:
- Encouraging rolling skills
- Enhancing stability while sitting
- Promoting pulling up to stand
- Encouraging crawling

### 5-Day Fun and Game-Like Home Plan

**Day 1: Tummy Time Treasure Hunt**
- **Activity:** Turn tummy time into a treasure hunt! Spread colorful toys around her while she’s on her tummy, encouraging Emma to reach for them. A friend or sibling can cheer her on.
- **Goal:** Strengthen her neck and shoulder muscles while making reaching enjoyable.

**Day 2: Rolling Race**
- **Activity:** Set up a rolling race! Use a bright toy that moves or rolls (like a ball) and have Emma roll from tummy to back to follow it. Take turns to make it a fun competition.
- **Goal:** Enhance her rolling skills while adding excitement.

**Day 3: Sit and Reach Game**
- **Activity:** Create a sit-an

In [27]:
events = await plan_runner.run_debug(
    "Please shorten Day 2 to just ONE activity. I’m busy.",
    user_id="emma_parents",
    session_id=session_id,   # same session where plan was created
    quiet=True,
    verbose=False,
)
print_last_model_message(events)

Event from an unknown agent: milestone_qna_agent_gross_motor, event id: 4cd02808-5ce9-4330-bf08-d96beff1c4b8
Event from an unknown agent: milestone_qna_agent_gross_motor, event id: a4c2568c-6817-434f-90e7-9617e120656f
Event from an unknown agent: milestone_qna_agent_gross_motor, event id: 872b138e-3a80-44f6-8a8a-aee1515d44dd
Event from an unknown agent: delay_agent_gross_motor, event id: 70671acd-68cd-4375-ab42-3a5652f8e8bc
Event from an unknown agent: profile_agent_gross_motor, event id: fad89639-a01b-4529-9676-16bb814621e2
Event from an unknown agent: profile_agent_gross_motor, event id: 4be549fd-3a24-44d2-8e65-a66789be1478
Event from an unknown agent: profile_agent_gross_motor, event id: c7ab5c8f-4ad0-4cc2-9dbc-78e75083fef7
Event from an unknown agent: milestone_qna_agent_gross_motor, event id: f80c084f-aa24-4eb8-9408-88fb1b61c818
Event from an unknown agent: milestone_qna_agent_gross_motor, event id: 6cfcf035-ba27-48ee-a068-b2d8f81cee94
Event from an unknown agent: milestone_qna_ag

**Emma is 3 years old and has Down syndrome.**

Emma's gross motor skills are estimated to be delayed by about 24 months, meaning she may function around the 12-month level.

### This week's focus:
- Encouraging rolling skills
- Enhancing stability while sitting
- Promoting pulling up to stand
- Encouraging crawling

### 5-Day Fun and Game-Like Home Plan

**Day 1: Tummy Time Treasure Hunt**
- **Activity:** Turn tummy time into a treasure hunt! Spread colorful toys around her while she’s on her tummy, encouraging Emma to reach for them. A friend or sibling can cheer her on.
- **Goal:** Strengthen her neck and shoulder muscles while making reaching enjoyable.

**Day 2: Rolling Race**
- **Activity:** Use a bright toy that rolls (like a ball) and encourage Emma to roll from tummy to back to follow it. Cheer her on and take turns to make it fun.
- **Goal:** Enhance her rolling skills while adding excitement to the activity.

**Day 3: Sit and Reach Game**
- **Activity:** Create a sit-and-rea